In [5]:
import numpy as np

import astropy
import astropy.units as ur
import astropy.constants as cr

%matplotlib inline

from astropy.io import fits
from specutils import Spectrum1D
from specutils.manipulation import (box_smooth, gaussian_smooth, trapezoid_smooth)

from pylab import *

import scipy.interpolate as II

In [6]:
fλ_unit = ur.erg/ur.s/ur.cm**2/ur.Angstrom # Spectral radiances per Hz or per angstrom
fν_unit = ur.erg/ur.s/ur.cm**2/ur.Hertz

Ω_arcsecond = 1/(3600.**2 * 57.4**2) * ur.sr

µ = ur.micron
arcsec = ur.arcsec
nm = ur.nm

In [7]:
def fnum_to_steradian(fnum):
    " See eq. 24 https://www.labsphere.com/site/assets/files/2551/integrating_sphere_theory_apps_tech_guide.pdf"
    
    return np.pi/(4*fnum*fnum) * ur.steradian


def steradian_to_fnum(sr):
    
    return np.sqrt(np.pi/(4*sr/ur.steradian))

## Inputs

In [8]:
pixel_size = 10 * µ
plate_scale = 6 * arcsec
RN = 10
BGDs = np.array([22, 22, 22]) * ur.ABmag
target = 22 * ur.ABmag
exptime = 300*ur.second
efficiency = 0.4
num_pixels_psf = 25

bands = np.array([ 
    (180, 220),
    (240, 300),
    (340, 380)
        ])*nm

## Outputs

In [31]:
for bgd, band in zip(BGDs, bands):
    λ_mid = np.mean(band)
    epp = (cr.c*cr.h/λ_mid).cgs
    BW = np.abs(band[0]-band[1])
    
    A_pixel = pixel_size**2
    pixel_angular_area = (plate_scale**2 / ur.arcsecond**2).cgs
    
    # Spectral radiance in band --> # photons in band
    L_λ_bgd = bgd.to(fλ_unit, equivalencies=ur.spectral_density(λ_mid)) / Ω_arcsecond 
    L_λ_obj = target.to(fλ_unit, equivalencies=ur.spectral_density(λ_mid)) / Ω_arcsecond 
    
    num_photons_bgd = (L_λ_bgd * exptime * BW * efficiency * pixel_angular_area 
                       * A_pixel / epp).to(1/ur.steradian)
    num_photons_obj = (L_λ_obj * exptime * BW * efficiency * pixel_angular_area 
                       * A_pixel / epp / num_pixels_psf).to(1/ur.steradian)
    
    bgd_limit = 10 * RN
    required_Ω = bgd_limit / num_photons_bgd
    
    
    fnum = steradian_to_fnum(required_Ω)
    focal_length = (pixel_size/(plate_scale/ur.arcsec) * 206265).to(ur.mm)
    Dtel = (focal_length/fnum).to(ur.cm)
    
    print()
    print("At {0} the energy per photon is {1:1.1e}".format(λ_mid, epp))
    print("From {0} background the number of photons per pixel per Sr are {1:5.1e}".format(bgd, num_photons_bgd))
    print("From {0} target the number of photons per pixel per Sr are {1:5.1e}".format(target, num_photons_obj))
    print("Bgd Limit Arguments: f/{0:3.1f} with f={1:3.1f} ∴ Dtel={2:3.1f}".format(fnum, focal_length, Dtel))
    
    required_Ω_snr = 2000 / num_photons_obj
    nobj = num_photons_obj*required_Ω_snr*num_pixels_psf
    nbgd = num_photons_bgd*required_Ω_snr*num_pixels_psf
    
    SNR = nobj/np.sqrt(nobj + nbgd + RN**2*num_pixels_psf)
    

    fnum = steradian_to_fnum(required_Ω_snr)
    focal_length = (pixel_size/(plate_scale/ur.arcsec) * 206265).to(ur.mm)
    Dtel = (focal_length/fnum).to(ur.cm)
    
    print("SNR Arguments: f/{0:3.1f} with f={1:3.1f} ∴ Dtel={2:3.1f}".format(fnum, focal_length, Dtel))
    print("SNR of target is {0:3.1f}".format(SNR))
    print(nobj, nbgd, num_photons_obj/num_photons_bgd, required_Ω_snr)


At 200.0 nm the energy per photon is 9.9e-12 erg
From 22.0 mag(AB) background the number of photons per pixel per Sr are 3.2e+05 1 / sr
From 22.0 mag(AB) target the number of photons per pixel per Sr are 1.3e+04 1 / sr
Bgd Limit Arguments: f/50.2 with f=343.8 mm ∴ Dtel=0.7 cm
SNR Arguments: f/2.2 with f=343.8 mm ∴ Dtel=15.3 cm
SNR of target is 43.8
50000.0 1249999.9999999998 0.04 0.15605708762593035 sr

At 270.0 nm the energy per photon is 7.4e-12 erg
From 22.0 mag(AB) background the number of photons per pixel per Sr are 3.6e+05 1 / sr
From 22.0 mag(AB) target the number of photons per pixel per Sr are 1.4e+04 1 / sr
Bgd Limit Arguments: f/52.9 with f=343.8 mm ∴ Dtel=0.7 cm
SNR Arguments: f/2.4 with f=343.8 mm ∴ Dtel=14.5 cm
SNR of target is 43.8
49999.99999999999 1250000.0 0.039999999999999994 0.1404513788633373 sr

At 360.0 nm the energy per photon is 5.5e-12 erg
From 22.0 mag(AB) background the number of photons per pixel per Sr are 1.8e+05 1 / sr
From 22.0 mag(AB) target the numb